In [1]:
import pandas as pd 
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [9]:
TEXT_chicken = open('chicken.txt', 'rb').read().decode(encoding='utf-8')
TEXT_meat = open('meat.txt', 'rb').read().decode(encoding='utf-8')
TEXT_fish = open('fish.txt', 'rb').read().decode(encoding='utf-8')
TEXT_ingredients = open('ingredients.txt', 'rb').read().decode(encoding='utf-8')
# The length of texts is the number of characters in it
print(len(TEXT_chicken))
print(len(TEXT_meat))
print(len(TEXT_fish))
print(len(TEXT_ingredients))

500000
500000
500000
10000


In [3]:

# The unique characters in the file
vocab = sorted(set(TEXT_fish))
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

TEXT_as_int = np.array([char2idx[c] for c in TEXT_fish])
seq_length = 100
examples_per_epoch = len(TEXT_fish)//(seq_length+1)


# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(TEXT_as_int)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

def split_input_target(chunk):
    input_TEXT = chunk[:-1]
    target_TEXT = chunk[1:]
    return input_TEXT, target_TEXT

dataset = sequences.map(split_input_target)
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)


# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")


def generate_text(model, start_string,t):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 1000

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = t

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))


def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)


example_batch_loss = loss(target_example_batch, example_batch_predictions)
model.compile(optimizer='adam', loss=loss)

# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)
EPOCHS = 35

history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])
model.save("fish_model.h5")


model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))


text_file_f = generate_text(model, start_string=u" ",t=0.1)


(64, 100, 40) # (batch_size, sequence_length, vocab_size)
Epoch 1/35
77/77 [==============================] - 13s 131ms/step - loss: 2.8172
Epoch 2/35
77/77 [==============================] - 11s 131ms/step - loss: 1.9814
Epoch 3/35
77/77 [==============================] - 11s 129ms/step - loss: 1.5906
Epoch 4/35
77/77 [==============================] - 11s 130ms/step - loss: 1.3147
Epoch 5/35
77/77 [==============================] - 11s 132ms/step - loss: 1.1492
Epoch 6/35
77/77 [==============================] - 11s 130ms/step - loss: 1.0402
Epoch 7/35
77/77 [==============================] - 11s 131ms/step - loss: 0.9619
Epoch 8/35
77/77 [==============================] - 11s 132ms/step - loss: 0.8966
Epoch 9/35
77/77 [==============================] - 11s 132ms/step - loss: 0.8438
Epoch 10/35
77/77 [==============================] - 11s 130ms/step - loss: 0.7963
Epoch 11/35
77/77 [==============================] - 11s 130ms/step - loss: 0.7503
Epoch 12/35
77/77 [===================

In [4]:
text_file_f

' and lightly beat egg with 1 tablespoon olive oil. sprinkle with 14 teaspoon sea salt and 14 teaspoon pepper. spread the bread on a baking sheet, spray the bread wedges with olive oil spray., fill a medium saucepan with water and bring to a boil over medium heat. add the onion, fennel, leek, peppercorns, salt, parsley and bay leaf. turn the heat to medium and simmer until the chicken is cooked through, about 10 minutes. drain and transfer the chicken to a cutting board strain the crushed tomatoes, fish stock, seaweed, hot italian peppers, crushed red pepper and season with salt and pepper. spread tapenade on top of steaks. pour orange juice and wine over and around the fish and shrimp. heat olive oil in large nonstick frying pan and add the minced onion. cook until it begins to soften and add the garlic, tomato, and parsley. cook, stirring occasionally, until soft and lightly caramelized. chill in the refrigerator for 1 hour., add the reduced juice mixture to the bowl along with the s

In [5]:

# The unique characters in the file
vocab = sorted(set(TEXT_meat))
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

TEXT_as_int = np.array([char2idx[c] for c in TEXT_meat])
seq_length = 100
examples_per_epoch = len(TEXT_meat)//(seq_length+1)


# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(TEXT_as_int)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

def split_input_target(chunk):
    input_TEXT = chunk[:-1]
    target_TEXT = chunk[1:]
    return input_TEXT, target_TEXT

dataset = sequences.map(split_input_target)
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)


# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")


def generate_text(model, start_string,t):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 1000

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = t

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))


def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)


example_batch_loss = loss(target_example_batch, example_batch_predictions)
model.compile(optimizer='adam', loss=loss)

# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)
EPOCHS = 35

history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])
model.save("meat_model.h5")


model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))


text_file_m = generate_text(model, start_string=u" ",t=0.1)


(64, 100, 40) # (batch_size, sequence_length, vocab_size)
Epoch 1/35
77/77 [==============================] - 13s 134ms/step - loss: 2.7921
Epoch 2/35
77/77 [==============================] - 11s 131ms/step - loss: 1.9829
Epoch 3/35
77/77 [==============================] - 11s 132ms/step - loss: 1.5716
Epoch 4/35
77/77 [==============================] - 11s 131ms/step - loss: 1.2940
Epoch 5/35
77/77 [==============================] - 11s 131ms/step - loss: 1.1354
Epoch 6/35
77/77 [==============================] - 11s 132ms/step - loss: 1.0380
Epoch 7/35
77/77 [==============================] - 11s 131ms/step - loss: 0.9699
Epoch 8/35
77/77 [==============================] - 11s 131ms/step - loss: 0.9172
Epoch 9/35
77/77 [==============================] - 11s 132ms/step - loss: 0.8737
Epoch 10/35
77/77 [==============================] - 11s 131ms/step - loss: 0.8368
Epoch 11/35
77/77 [==============================] - 11s 131ms/step - loss: 0.8056
Epoch 12/35
77/77 [===================

In [6]:
text_file_m

' for 20 seconds or pour some of the beef stock to come up 23 of the tomatoes with a sharp pairing knife. once the water comes to a boil, reduce to a simmer and cook until slightly browned. add the stock and meat to the pot and cook, stirring occasionally, until the onions are lightly browned and the coriander is fragrant, about 10 seconds per side. transfer the cooked burgers to a rimmed baking sheet. sprinkle with the parsley. season the soup with salt and pepper. serve with toppings, as desired., classic stove top chili recipe, heat a large pot on medium heat, add the garlic and cook until golden brown on both sides, about 30 minutes. let stand 5 minutes. season with salt and pepper. serve with toppings, as desired., classic stove top chili recipe, heat a large pot on medium heat, add onions and garlic to a small bowl. add more olive oil to the pan to melt. sear the veal shanks, turning carefully with tongs, until all sides are a rich brown caramel color. drizzle with a little more 

In [7]:

# The unique characters in the file
vocab = sorted(set(TEXT_chicken))
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

TEXT_as_int = np.array([char2idx[c] for c in TEXT_chicken])
seq_length = 100
examples_per_epoch = len(TEXT_chicken)//(seq_length+1)


# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(TEXT_as_int)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

def split_input_target(chunk):
    input_TEXT = chunk[:-1]
    target_TEXT = chunk[1:]
    return input_TEXT, target_TEXT

dataset = sequences.map(split_input_target)
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)


# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")


def generate_text(model, start_string,t):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 1000

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = t

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))


def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)


example_batch_loss = loss(target_example_batch, example_batch_predictions)
model.compile(optimizer='adam', loss=loss)

# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)
EPOCHS = 35

history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])
model.save("chicken_model.h5")


model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))


text_file_c = generate_text(model, start_string=u" ",t=0.1)


(64, 100, 40) # (batch_size, sequence_length, vocab_size)
Epoch 1/35
77/77 [==============================] - 12s 131ms/step - loss: 2.8139
Epoch 2/35
77/77 [==============================] - 11s 130ms/step - loss: 1.9306
Epoch 3/35
77/77 [==============================] - 11s 129ms/step - loss: 1.5130
Epoch 4/35
77/77 [==============================] - 11s 131ms/step - loss: 1.2404
Epoch 5/35
77/77 [==============================] - 11s 130ms/step - loss: 1.0877
Epoch 6/35
77/77 [==============================] - 11s 131ms/step - loss: 0.9941
Epoch 7/35
77/77 [==============================] - 11s 130ms/step - loss: 0.9298
Epoch 8/35
77/77 [==============================] - 11s 133ms/step - loss: 0.8821
Epoch 9/35
77/77 [==============================] - 11s 131ms/step - loss: 0.8404
Epoch 10/35
77/77 [==============================] - 11s 131ms/step - loss: 0.8065
Epoch 11/35
77/77 [==============================] - 11s 132ms/step - loss: 0.7762
Epoch 12/35
77/77 [===================

In [8]:
text_file_c

' minutes. remove to a baking sheet and bake in the preheated oven for 10 minutes. remove from heat and stir in parsley. put potato through cavity along with the remaining 12 teaspoon salt. cover and set aside., for chicken, preheat outdoor grill or grill pan over mediumhigh heat. add the onions and saute until soft, about 5 minutes. stir in the marinade and transfer the remainder to the flour, stirring occasionally. add the peas and cilantro and warm through. garnish with sour cream and whole or chopped cilantro and serve., combine all measured ingredients in a small saucepan and heat the bread crumbs and panko. dip chicken into egg wash, then breadcrumbs and transfer to a sheet pan with oil, bake 35 minutes, until the peas are tender., remove from oven and top with fresh parsley.for the chicken, put the olive oil in a heavybottomed pot over medium heat. add the onions and saute until soft, about 5 minutes. stir in the mint and lemon zest and sprinkle with the allspice. season with sa

In [34]:
import re
def word_extraction(sentence):
  ignore = ["a", "the", "is", "to", "until","about","with","and","as","add","on","in","at","of","over","around","knife","sharp","pairing","once","comes","for","minutes","hours"]
  words = re.sub("[^\w]", " ",  sentence).split()
  cleaned_text = [w.lower() for w in words if w not in ignore]
  return str(cleaned_text)


ingredients_m = word_extraction(text_file_m)
ingredients_c = word_extraction(text_file_c)
ingredients_f = word_extraction(text_file_f)
##


In [35]:
def chicken_meal_generator():
  output = [ingredients_c, text_file_c]
  return output

def fish_meal_generator():
  output = [ingredients_f, text_file_f]
  return output

def meat_meal_generator():
  output = [ingredients_m, text_file_m]
  return output

In [36]:
meat_meal_generator()

["['20', 'seconds', 'or', 'pour', 'some', 'beef', 'stock', 'come', 'up', '23', 'tomatoes', 'water', 'boil', 'reduce', 'simmer', 'cook', 'slightly', 'browned', 'stock', 'meat', 'pot', 'cook', 'stirring', 'occasionally', 'onions', 'are', 'lightly', 'browned', 'coriander', 'fragrant', '10', 'seconds', 'per', 'side', 'transfer', 'cooked', 'burgers', 'rimmed', 'baking', 'sheet', 'sprinkle', 'parsley', 'season', 'soup', 'salt', 'pepper', 'serve', 'toppings', 'desired', 'classic', 'stove', 'top', 'chili', 'recipe', 'heat', 'large', 'pot', 'medium', 'heat', 'garlic', 'cook', 'golden', 'brown', 'both', 'sides', '30', 'let', 'stand', '5', 'season', 'salt', 'pepper', 'serve', 'toppings', 'desired', 'classic', 'stove', 'top', 'chili', 'recipe', 'heat', 'large', 'pot', 'medium', 'heat', 'onions', 'garlic', 'small', 'bowl', 'more', 'olive', 'oil', 'pan', 'melt', 'sear', 'veal', 'shanks', 'turning', 'carefully', 'tongs', 'all', 'sides', 'are', 'rich', 'brown', 'caramel', 'color', 'drizzle', 'little',

In [37]:
fish_meal_generator()

["['lightly', 'beat', 'egg', '1', 'tablespoon', 'olive', 'oil', 'sprinkle', '14', 'teaspoon', 'sea', 'salt', '14', 'teaspoon', 'pepper', 'spread', 'bread', 'baking', 'sheet', 'spray', 'bread', 'wedges', 'olive', 'oil', 'spray', 'fill', 'medium', 'saucepan', 'water', 'bring', 'boil', 'medium', 'heat', 'onion', 'fennel', 'leek', 'peppercorns', 'salt', 'parsley', 'bay', 'leaf', 'turn', 'heat', 'medium', 'simmer', 'chicken', 'cooked', 'through', '10', 'drain', 'transfer', 'chicken', 'cutting', 'board', 'strain', 'crushed', 'tomatoes', 'fish', 'stock', 'seaweed', 'hot', 'italian', 'peppers', 'crushed', 'red', 'pepper', 'season', 'salt', 'pepper', 'spread', 'tapenade', 'top', 'steaks', 'pour', 'orange', 'juice', 'wine', 'fish', 'shrimp', 'heat', 'olive', 'oil', 'large', 'nonstick', 'frying', 'pan', 'minced', 'onion', 'cook', 'it', 'begins', 'soften', 'garlic', 'tomato', 'parsley', 'cook', 'stirring', 'occasionally', 'soft', 'lightly', 'caramelized', 'chill', 'refrigerator', '1', 'hour', 'red

In [38]:
chicken_meal_generator()

["['remove', 'baking', 'sheet', 'bake', 'preheated', 'oven', '10', 'remove', 'from', 'heat', 'stir', 'parsley', 'put', 'potato', 'through', 'cavity', 'along', 'remaining', '12', 'teaspoon', 'salt', 'cover', 'set', 'aside', 'chicken', 'preheat', 'outdoor', 'grill', 'or', 'grill', 'pan', 'mediumhigh', 'heat', 'onions', 'saute', 'soft', '5', 'stir', 'marinade', 'transfer', 'remainder', 'flour', 'stirring', 'occasionally', 'peas', 'cilantro', 'warm', 'through', 'garnish', 'sour', 'cream', 'whole', 'or', 'chopped', 'cilantro', 'serve', 'combine', 'all', 'measured', 'ingredients', 'small', 'saucepan', 'heat', 'bread', 'crumbs', 'panko', 'dip', 'chicken', 'into', 'egg', 'wash', 'then', 'breadcrumbs', 'transfer', 'sheet', 'pan', 'oil', 'bake', '35', 'peas', 'are', 'tender', 'remove', 'from', 'oven', 'top', 'fresh', 'parsley', 'chicken', 'put', 'olive', 'oil', 'heavybottomed', 'pot', 'medium', 'heat', 'onions', 'saute', 'soft', '5', 'stir', 'mint', 'lemon', 'zest', 'sprinkle', 'allspice', 'seas